In [1]:
import numpy as np 
import torch 
import matplotlib.pyplot as plt 

from mpol import coordinates
from mpol.gridding import DirtyImager

In [2]:
# Observation
data = np.load("../../HTLup_continuum2.npz")
u = data["uu"] # klambda units
v = data["vv"]
vis = data["data"]
#wavelength = data["wavelength"]
weight = data["weight"]

# Loading the psf and the dirty image
pixel_scale = 0.0015 # arcsec
npix = 256 # Number of pixels in the grid
coords = coordinates.GridCoords(cell_size=pixel_scale, npix=npix)